<a href="https://colab.research.google.com/github/karanbhatt1/MachineLearning/blob/main/Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing neccessary libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
#header = null cleared the irrelavent colmn name
# name = [name of cloumns]
# usecols = [where to put the columns name];
#dataset loading using pandas
netflix_dataset = pd.read_csv("/content/drive/MyDrive/NetflixProject/Copy of combined_data_1.txt.zip",header=None,names=['cust_id','Ratings'],usecols=[0,1])

In [ ]:
netflix_dataset.head(100)

In [ ]:
netflix_dataset.isna().sum().sum()

In [ ]:
movie_count = netflix_dataset['Ratings'].isna().sum()
movie_count

In [ ]:
netflix_dataset['cust_id'].value_counts()

In [ ]:
customer_count = netflix_dataset['cust_id'].nunique() - movie_count

In [ ]:
no_of_ratings = netflix_dataset['cust_id'].count() - movie_count

In [ ]:
no_of_ratings

In [ ]:
netflix_dataset.columns

In [ ]:
stars = netflix_dataset['Ratings'].value_counts()

In [ ]:
stars

In [ ]:
stars = netflix_dataset.groupby(netflix_dataset['Ratings']).count()

In [ ]:
stars

In [ ]:
movie_id = None
lis = []
for i in netflix_dataset['cust_id']:
  if ":" in i :
    movie_id = int(i.replace(":",""))
  lis.append(movie_id)

In [ ]:
lis

In [ ]:
netflix_dataset["Movie_Id"] = lis

In [ ]:
netflix_dataset

In [ ]:
netflix_dataset.dropna(axis=0,inplace=True)


In [ ]:
netflix_dataset

In [ ]:
netflix_dataset.dtypes

In [ ]:
#converting to numeric datatype
netflix_dataset['cust_id']= pd.to_numeric(netflix_dataset['cust_id']) # astype can also be used

In [ ]:
netflix_dataset.info()

In [ ]:
movie_re_count = netflix_dataset['Movie_Id'].value_counts()

In [ ]:
movie_re_count # A particular movie had been reviewed how many times

In [ ]:
#isin ==> returns a bollean value if its present in other data

In [ ]:
bench_mark_movie = round(movie_re_count.quantile(0.6))

In [ ]:
bench_mark_movie # A threshold value

In [ ]:
#the customer who had reviewed more than threshold movie count should be taken
cust_rew_count  = netflix_dataset['cust_id'].value_counts()

In [ ]:
bench_mark_cust= round(cust_rew_count.quantile(0.6))
bench_mark_cust

In [ ]:
drop_movies = movie_re_count[movie_re_count<=bench_mark_movie].index # .index function convert the dataframe into array

In [86]:
drop_movies #need to drop from the dataset;

Index([ 929, 1598, 1733, 1647, 4099, 1616, 1446,  263, 4259,  160,
       ...
       1858, 4035, 3693, 2805,  820, 4294,  915, 3656, 4338, 4362],
      dtype='int64', name='Movie_Id', length=2700)

In [ ]:
drop_cust = cust_rew_count[cust_rew_count<=bench_mark_cust].index
drop_cust

In [ ]:
netflix_dataset.columns

In [ ]:
netflix_dataset = netflix_dataset.loc[~netflix_dataset['Movie_Id'].isin(drop_movies)] # taking the data in which more the movie is reviwed more than the threshold values
netflix_dataset = netflix_dataset.loc[~netflix_dataset['cust_id'].isin(drop_cust)]  # taking the data in which customer has reviewed movies more than threshold values;

In [ ]:
netflix_dataset

In [ ]:
movie_data = pd.read_csv("//content//drive//MyDrive//NetflixProject//Copy of movie_titles.csv",encoding='ISO-8859-1',header=None,names=['Movie_Id','Year','Name'],usecols=[0,1,2])

In [ ]:
movie_data.head()

In [ ]:
!pip install scikit-surprise

In [ ]:
!pip install numpy==1.23.5

In [ ]:
from surprise import SVD,Reader,Dataset
from surprise.model_selection import cross_validate

In [ ]:
reader = Reader() # used to read the data from dataset

In [ ]:
#entire data store in the object data:
data = Dataset.load_from_df(netflix_dataset[['cust_id', 'Movie_Id', 'Ratings']][:10001],reader)

In [ ]:
data

In [ ]:
model = SVD()

In [ ]:
#Matrix Decomposition ==> dividing the matrix into parts .
# row * col
# reduce search range.

#inside model SVD function is SVD (support vector decomposition);

In [ ]:
cross_validate(model,data,measures=['RMSE'],cv=6,verbose=True)

In [ ]:
netflix_dataset['cust_id']

In [ ]:
user_rating =  netflix_dataset[netflix_dataset['cust_id']==2643029]
user_2643029 = movie_data.copy()

In [103]:
user_rating

,cust_id,Ratings,Movie_Id
211028,2643029,4.0,30
539366,2643029,4.0,156
815159,2643029,3.0,191
1062501,2643029,5.0,241
1453806,2643029,4.0,299
...,...,...,...
22869123,2643029,4.0,4306
23291445,2643029,5.0,4356
23829756,2643029,4.0,4454
24018591,2643029,4.0,4488


In [ ]:
user_rating = netflix_dataset.pivot_table(index='cust_id',columns='Movie_Id',values='Ratings')

In [ ]:
user_2643029 = user_2643029[~user_2643029['Movie_Id'].isin(drop_movies)]

In [ ]:
user_2643029

In [88]:
user_2643029['Estimated'] = user_2643029['Movie_Id'].apply(lambda x: model.predict(2643029,x).est)

<ipython-input-88-d315f9c907a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_2643029['Estimated'] = user_2643029['Movie_Id'].apply(lambda x: model.predict(2643029,x).est)


In [89]:
user_2643029.sort_values("Estimated",ascending=False);

In [91]:
user_2643029 # the movie having the highest estimated score calulated from 5 using the svd

,Movie_Id,Year,Name,Estimated
2,3,1997.0,Character,3.667924
4,5,2004.0,The Rise and Fall of ECW,3.890563
5,6,1997.0,Sick,3.105926
7,8,2004.0,What the #$*! Do We Know!?,3.151568
15,16,1996.0,Screamers,3.287702
...,...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...,3.287702
17766,17767,2004.0,Fidel Castro: American Experience,3.287702
17767,17768,2000.0,Epoch,3.287702
17768,17769,2003.0,The Company,3.287702


In [95]:
netflix_dataset['cust_id']

,cust_id
696,712664
697,1331154
698,2632461
699,44937
700,656399
...,...
24056842,1055714
24056843,2643029
24056844,267802
24056845,1559566


## Created Function to take customer id from the user and recommend the movies


In [123]:
def user():
  userid = int(input("enter the user id"))
  data = netflix_dataset[netflix_dataset['cust_id']==userid]
  if(data.value_counts().sum()==0):
    print("user not found")
  else:
     print(data)

enter the user id89756
user not found


In [147]:
def recomendationByUser(userid):
  user = movie_data.copy()
  user = user[~user['Movie_Id'].isin(drop_movies)]
  user['Estimated'] = user['Movie_Id'].apply(lambda x: model.predict(userid,x).est)
  return user.head(10)


In [151]:
userid = int(input("enter the user id"))
data = netflix_dataset[netflix_dataset['cust_id']==userid]
if(data.value_counts().sum()==0):
  print("user not found")
else:
  print(pd.DataFrame(recomendationByUser(userid)))

enter the user id1559566
    Movie_Id    Year                                           Name  Estimated
2          3  1997.0                                      Character   3.667924
4          5  2004.0                       The Rise and Fall of ECW   3.890563
5          6  1997.0                                           Sick   3.105926
7          8  2004.0                     What the #$*! Do We Know!?   3.151568
15        16  1996.0                                      Screamers   3.287702
16        17  2005.0                                      7 Seconds   3.287702
17        18  1994.0                               Immortal Beloved   3.287702
23        24  1981.0                            My Bloody Valentine   3.287702
24        25  1997.0  Inspector Morse 31: Death Is Now My Neighbour   3.287702
25        26  2004.0                                Never Die Alone   3.287702
